# Matrix and compartive statistics review

The following notebook is a review of matrices and comparative statistics with examples in python.

The equations and examples are from the following book I highly recommend using to brush up on mathamtics commonly used in economics coursework:
- Dowling, E. T. (2012). Introduction to mathematical economics. McGraw-Hill.
    - [Amazon link](https://www.amazon.com/Schaums-Introduction-Mathematical-Economics-Outlines/dp/0071762515/ref=sr_1_7?dchild=1&keywords=mathematics+economics&qid=1593200726&sr=8-7)

# Table of contents
- [1. Matrix basics](#1.-Matrix-basics)
    
- [2. Special determinants](#2.-Special-determinants)

- [3. Comparative statistics](#3.-Comparative-statistics)



# 1. Matrix basics

In [1]:
import numpy as np
np.random.seed(1)

## 1.1 Scalar multiplication

In [2]:
A = np.random.randint(20, size=(2,2))
A

array([[ 5, 11],
       [12,  8]])

In [3]:
A*3

array([[15, 33],
       [36, 24]])

## 1.2 Matrix addition

In [4]:
A = np.random.randint(20, size=(2,2))
B = np.random.randint(20, size=(2,2))
A+B

array([[ 9, 27],
       [ 6, 27]])

## 1.3 Matrix mulitiplication

In [5]:
A = np.random.randint(20, size=(2,2))
B = np.random.randint(20, size=(2,2))

In [6]:
A@B

array([[178, 256],
       [228, 288]])

### 1.4 Identity matrix and empty

In [7]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [8]:
np.zeros(shape=(2,2))

array([[0., 0.],
       [0., 0.]])

## 1.5 Matrix inversion

In [9]:
A = np.random.randint(20, size=(2,2))
A

array([[14, 18],
       [ 4,  9]])

In [11]:
#Determinate
round(np.linalg.det(A))

54.0

In [15]:
#Invert matrix
np.linalg.inv(A).round(3)

array([[ 0.167, -0.333],
       [-0.074,  0.259]])

# 2. Special determinants

## 2.1 Jacobian

In [32]:
import sympy as sy
x1, x2 = sy.symbols('x1 x2', integer=True)
y1 = 5*x1+3*x2
y2 = 25*x1**2+30*x1*x2+9*x2**2

In [33]:
y1

5*x1 + 3*x2

In [34]:
y2

25*x1**2 + 30*x1*x2 + 9*x2**2

In [35]:
independent_variables = [x1, x2]
functions = [y1, y2]
Jacobian = sy.Matrix(np.zeros(shape=(len(functions), len(independent_variables))))
count = 0
for funcs in functions:
    for iv in independent_variables:
        Jacobian[count] = sy.diff(funcs, iv)
        count+=1
Jacobian

Matrix([
[            5,             3],
[50*x1 + 30*x2, 30*x1 + 18*x2]])

In [37]:
if Jacobian.det()==0:
    print("Functional dependence")

Functional dependence


## 2.2 Hessian

### 2.2.1 Sympy example: Definition

In [38]:
from sympy import Function, hessian
from sympy.abc import x, y
f = Function('f')(x, y)
hessian(f, (x, y))

Matrix([
[Derivative(f(x, y), (x, 2)),   Derivative(f(x, y), x, y)],
[  Derivative(f(x, y), x, y), Derivative(f(x, y), (y, 2))]])

### 2.2.1 Sympy example: From 2.1

In [39]:
count = 0
Hessian = Jacobian.copy()
for _ in range(0,2):
    for iv in independent_variables: #Reverses list order for hessian
        Hessian[count] = sy.diff(Hessian[count],iv)
        count+=1
Hessian

Matrix([
[ 0,  0],
[50, 18]])

In [40]:
H1 = Hessian[0]
H2 = Hessian.det()
if H1>0 and H2>0:
    print('Positive definite')
    print('Minimum point')
if H1<0 and H2>0:
    print('Negative definite')
    print('Max point')

## 2.3 Discriminant
- Tests for positive or negative definiteness of quadratic equations

In [41]:
x, y = sy.symbols('x y', integer=True)
z = 2*x**2 + 5*x*y+8*y**2
z

2*x**2 + 5*x*y + 8*y**2

In [42]:
Discrim = sy.Matrix([[2,(5/2)], [(5/2),8]])
Discrim 

Matrix([
[  2, 2.5],
[2.5,   8]])

In [43]:
d1 = Discrim[0]
d2 = Discrim.det() #deter
if d1 and d2>0:
    print("Positive definite")
elif d1<0 and d2>0:
    print('Negative definite')

Positive definite


## 2.4 Higher order hessian

In [44]:
x1, x2, x3 = sy.symbols('x1 x2 x3', integer=True)
z = -5*x1**2+10*x1+x1*x3-2*x2**2+4*x2+2*x2*x3-4*x3**2
z

-5*x1**2 + x1*x3 + 10*x1 - 2*x2**2 + 2*x2*x3 + 4*x2 - 4*x3**2

In [45]:
focs = []
for idx,iv in enumerate([x1,x2,x3]):
    foc = sy.diff(z,iv)
    focs.append(foc)
for idx, foc in enumerate(focs):
    print("FOC %s:" %(idx+1), foc)

FOC 1: -10*x1 + x3 + 10
FOC 2: -4*x2 + 2*x3 + 4
FOC 3: x1 + 2*x2 - 8*x3


In [46]:
A,b = sy.linear_eq_to_matrix(focs, [x1, x2, x3])

In [47]:
Hessian = A
Hessian

Matrix([
[-10,  0,  1],
[  0, -4,  2],
[  1,  2, -8]])

In [48]:
H1 = Hessian[0]
H2 = Hessian[0:2,0:2].det()
H3 = Hessian.det()
if H1>0 and H2>0 and H3>0:
    print('Positive definite')
    print('Minimum point')
elif H1<0 and H2>0 and H3<0:
    print('Negative definite')
    print('Maximum point')

Negative definite
Maximum point


## 2.5 Bordered Hessian

In [49]:
from sympy import Function, hessian, pprint
from sympy.abc import x, y
f = Function('f')(x, y)
constraint = Function('g')(x, y)
hessian(f, (x, y), [constraint])

Matrix([
[                     0,      Derivative(g(x, y), x),      Derivative(g(x, y), y)],
[Derivative(g(x, y), x), Derivative(f(x, y), (x, 2)),   Derivative(f(x, y), x, y)],
[Derivative(g(x, y), y),   Derivative(f(x, y), x, y), Derivative(f(x, y), (y, 2))]])

## 2.6 Eigenvalues & Eigenvectors

In [50]:
def eigen(matrix):
    trace = np.trace(A)
    det = round(np.linalg.det(A),0)
    eig_values = np.round((np.sort((trace+np.array([+1,-1])*np.sqrt(trace**2-(4*det)))/2)),1)
    solu1, solu2 = [eig_values[:][0], eig_values[:][1]]
    print("Original matrix: \n",matrix)
    print("Eigen-values:\n {}".format(eig_values))
    #Classification of matrix
    if solu1>0 and solu2>0:
        print('Pos definite')
    if solu1<0 and solu2<0:
        print('Neg definite')
    if (solu1==0 or solu2==0) and (solu1>0 and solu2>0):
        print('Pos semi-def')
    if (solu1==0 or solu2==0) and (solu1<0 and solu2<0):
        print('Pos semi-def')
    if (solu1<0 and solu2>0) or (solu1>0 and solu2<0):
        print('Indefinite')

In [51]:
A = np.random.randint(20, size=(2,2))
eigen(A)

Original matrix: 
 [[17  0]
 [13  9]]
Eigen-values:
 [ 9. 17.]
Pos definite


# 3. Comparative statistics

## 3.1 One endogenous variable

$$
Q_d = m-nP+kY\\
Q_s = a+bP
$$

### 3.1.1 Explicit function
$$P* =\frac{m-a+kY}{b+n}$$

### 3.1.2 Implicit function

$$\frac{dP^*}{dY}= - \frac{F_Y}{F_P}$$

In [52]:
from sympy.abc import x,n,p,k,y,a,b,m
f = m-n*p+k*y-a-b*p
f

-a - b*p + k*y + m - n*p

In [53]:
-sy.diff(f,y)/sy.diff(f,p)

-k/(-b - n)

## 3.2 N-endogenous variables
- `Comparative statistics:` requires a unique equilibrium condition for each endogenous variable
- Measuring the effect of an exogenous variable on the endgenous variables involves taking the total derivative of each equilibrium conditions
    - w.r.t to the particular exogenous variable and solving for each of the partial derivatives

$$
F^1(y_1, y_2; x_1, x_2) = 0 \\
F^2(y_1, y_2; x_1, x_2) = 0
$$

#### Note: 
- #### Exogenous variables: $x_1$ and $x_2$ 
- #### Endogenous variables: $y_1$ and $y_2$ 

## 3.3 Comparative statistics for optimization problems
- Apply comparative statistics to the first order conditions to determine initial optimal values

In [54]:
from sympy.abc import r, K, w, L, P, Q
Q = Function('Q')(K, L)
π = p*Q-r*K-w*L
π

-K*r - L*w + p*Q(K, L)

### 3.3.1 F.O.C

In [55]:
focs = []
for idx,iv in enumerate([K,L]):
    foc = sy.diff(π,iv)
    focs.append(foc)
focs[0]

p*Derivative(Q(K, L), K) - r

### 3.3.2 Jacobian

- For optimization of a system the Deteriminant of the Jacobian>0

In [63]:
Jacobian = sy.Matrix([[π.diff(K,K), π.diff(K,L)],[π.diff(L,K), π.diff(L,L)]])
Jacobian

Matrix([
[p*Derivative(Q(K, L), (K, 2)),   p*Derivative(Q(K, L), K, L)],
[  p*Derivative(Q(K, L), K, L), p*Derivative(Q(K, L), (L, 2))]])

In [64]:
B = []
for foc in focs:
    B.append(foc.diff(r))
B

[-1, 0]

### 3.3.3 Find derivatives

In [60]:
J = Jacobian.det()

In [67]:
J1 = Jacobian.copy()
J1[0] =1
J1[2] =0
J1

Matrix([
[1,   p*Derivative(Q(K, L), K, L)],
[0, p*Derivative(Q(K, L), (L, 2))]])

In [68]:
J2 = Jacobian.copy()
J2[1] =1
J2[3] =0
J2

Matrix([
[p*Derivative(Q(K, L), (K, 2)), 1],
[  p*Derivative(Q(K, L), K, L), 0]])

#### 3.3.3.1 Find $\frac{\partial \bar{K}}{\partial r}$

In [72]:
J1.det()/J

p*Derivative(Q(K, L), (L, 2))/(p**2*Derivative(Q(K, L), (K, 2))*Derivative(Q(K, L), (L, 2)) - p**2*Derivative(Q(K, L), K, L)**2)

#### 3.3.3.2 Find $\frac{\partial \bar{L}}{\partial r}$

In [73]:
J2.det()/J

-p*Derivative(Q(K, L), K, L)/(p**2*Derivative(Q(K, L), (K, 2))*Derivative(Q(K, L), (L, 2)) - p**2*Derivative(Q(K, L), K, L)**2)

## 3.4 Comparative statistics in constrained optimization
- Optimize comparative statistics with constraints

In [74]:
from sympy.abc import r, K, w, L, P, Q, B
lamda = sy.symbols('lamda')
Q = Function('Q')(K, L)
π = Q+lamda*(B-r*K-w*L)
π 

lamda*(B - K*r - L*w) + Q(K, L)

### 3.4.1 F.O.C

In [75]:
focs = []
for idx,iv in enumerate([K,L,lamda]):
    foc = sy.diff(π,iv)
    focs.append(foc)

In [76]:
focs[0]

-lamda*r + Derivative(Q(K, L), K)

In [77]:
focs[1]

-lamda*w + Derivative(Q(K, L), L)

In [78]:
focs[2]

B - K*r - L*w

### 3.4.2 Jacobian

In [79]:
independent_variables = [K,L,lamda]
functions = focs
Jacobian = sy.Matrix(np.zeros(shape=(len(functions), len(independent_variables))))
count = 0
for funcs in functions:
    for iv in independent_variables:
        Jacobian[count] = sy.diff(funcs, iv)
        count+=1
Jacobian

Matrix([
[Derivative(Q(K, L), (K, 2)),   Derivative(Q(K, L), K, L), -r],
[  Derivative(Q(K, L), K, L), Derivative(Q(K, L), (L, 2)), -w],
[                         -r,                          -w,  0]])

In [312]:
J_deter = Jacobian.det()

### 3.4.3 Find derivatives


In [308]:
def deriv_convert(matrix, col=0):
    deriv_col = iter([0, 0, -1])
    derivs = Jacobian.copy()
    for i in range(1,len(Jacobian)+1):
        if i%3==col:
            derivs[i-1] = next(deriv_col)
    return derivs

#### 3.3.4.1 Find $\frac{\partial \bar{K}}{\partial B}$

In [309]:
k_b = deriv_convert(derivs, col=1)
k_b

Matrix([
[ 0,   Derivative(Q(K, L), K, L), -r],
[ 0, Derivative(Q(K, L), (L, 2)), -w],
[-1,                          -w,  0]])

In [314]:
k_b.det()/J_deter

(-r*Derivative(Q(K, L), (L, 2)) + w*Derivative(Q(K, L), K, L))/(-r**2*Derivative(Q(K, L), (L, 2)) + 2*r*w*Derivative(Q(K, L), K, L) - w**2*Derivative(Q(K, L), (K, 2)))

#### 3.3.4.2 Find $\frac{\partial \bar{L}}{\partial B}$

In [310]:
L_b = deriv_convert(derivs, col=2)
L_b

Matrix([
[Derivative(Q(K, L), (K, 2)),  0, -r],
[  Derivative(Q(K, L), K, L),  0, -w],
[                         -r, -1,  0]])

In [315]:
L_b.det()/J_deter

(r*Derivative(Q(K, L), K, L) - w*Derivative(Q(K, L), (K, 2)))/(-r**2*Derivative(Q(K, L), (L, 2)) + 2*r*w*Derivative(Q(K, L), K, L) - w**2*Derivative(Q(K, L), (K, 2)))

#### 3.3.4.3 Find $\frac{\partial \bar{\lambda}}{\partial B}$

In [311]:
lamb_b = deriv_convert(derivs, col=0)
lamb_b 

Matrix([
[Derivative(Q(K, L), (K, 2)),   Derivative(Q(K, L), K, L),  0],
[  Derivative(Q(K, L), K, L), Derivative(Q(K, L), (L, 2)),  0],
[                         -r,                          -w, -1]])

In [316]:
lamb_b.det()/J_deter

(-Derivative(Q(K, L), (K, 2))*Derivative(Q(K, L), (L, 2)) + Derivative(Q(K, L), K, L)**2)/(-r**2*Derivative(Q(K, L), (L, 2)) + 2*r*w*Derivative(Q(K, L), K, L) - w**2*Derivative(Q(K, L), (K, 2)))

## 3.5 Envelope theorem

- `Envelope theorem:` Measures the effect of a change in exogenous variables on the optimal value of the objective function
    - This can be achieved by simply taking the derivative of the Lagrangian function w.r.t the desired exogenous variable and evaluating the derivative at the values of the optimal solution
    

In [343]:
B, x, px, y, py, lamb= sy.symbols('B x px y py lamda', integer=True)

u = Function('u')(x, y)
constraint = lamb*(B-px*x-py*y)
U = u+constraint
print('Budget constraint:')
U

Budget constraint:


lamda*(B - px*x - py*y) + u(x, y)

In [344]:
focs = []
for idx,iv in enumerate([px, py, B]):
    foc = sy.diff(U,iv)
    focs.append(foc)

- $\lambda$: Marginal uility of money
    - The extra utility derived from a change in income
    
- The first and second order conditions are negative:
    - $\uparrow \lambda \rightarrow$  negative impact on the quantity of good consumed

In [345]:
focs[0]

-lamda*x

In [346]:
focs[1]

-lamda*y

In [347]:
focs[2]

lamda

## 3.6 Concave programming
- Optimize comparative statistics with inequality constraints
- Assume that functions are concave

In [350]:
B, x, px, y, py, lamb= sy.symbols('B x px y py lamda', integer=True)

u = Function('u')(x, y)
constraint = lamb*(B-px*x-py*y)
U = u+constraint
print('Budget constraint:')
U

Budget constraint:


lamda*(B - px*x - py*y) + u(x, y)

In [351]:
#1.A
sy.diff(U, x)

-lamda*px + Derivative(u(x, y), x)

In [352]:
# 1.B
sy.diff(U, y)

-lamda*py + Derivative(u(x, y), y)

In [355]:
#2.A
sy.diff(U,lamb)

B - px*x - py*y